In [1]:
import os, sys
%pwd

'd:\\collab_projects\\Brain-Tumor-Detection-using-MLflow-DVC\\notebooks'

In [2]:
os.chdir("../")
%pwd

'd:\\collab_projects\\Brain-Tumor-Detection-using-MLflow-DVC'

In [3]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/gbiamgaurav/Brain-Tumor-Detection-using-MLflow-DVC.mlflow"
os.environ['MLFLOW_TRACKING_USERNAME'] = "gbiamgaurav"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "352861024ebede76aa75cf4ea021ee1f74b0e39f"

In [4]:
import tensorflow as tf

In [5]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from brain_tumor.constants import * 
from brain_tumor.utils.main_utils import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        evaluation_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/data",
            mlflow_uri="https://dagshub.com/gbiamgaurav/Brain-Tumor-Detection-using-MLflow-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return evaluation_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

d:\collab_projects\Brain-Tumor-Detection-using-MLflow-DVC\brain\lib\site-packages\pydantic\_internal\_fields.py:128: UserWarning: Field "model_server_url" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
d:\collab_projects\Brain-Tumor-Detection-using-MLflow-DVC\brain\lib\site-packages\pydantic\_internal\_config.py:317: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    

    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="Resnet50")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2023-10-03 14:37:43,314: INFO: main_utils: yaml file: config\config.yaml loaded successfully]
[2023-10-03 14:37:43,314: INFO: main_utils: yaml file: params.yaml loaded successfully]
[2023-10-03 14:37:43,314: INFO: main_utils: created directory at: artifacts]
Found 392 images belonging to 4 classes.
25/25 [==============================] - 25s 967ms/step - loss: 305.5476 - accuracy: 0.2296


2023/10/03 14:38:16 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: C:\Users\gaura\AppData\Local\Temp\tmp8p2l0eyd\model\data\model\assets
[2023-10-03 14:38:35,035: INFO: builder_impl: Assets written to: C:\Users\gaura\AppData\Local\Temp\tmp8p2l0eyd\model\data\model\assets]


d:\collab_projects\Brain-Tumor-Detection-using-MLflow-DVC\brain\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


[2023-10-03 14:40:03,323: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=5, redirect=5, status=5)) after connection broken by 'SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:2426)'))': /gbiamgaurav/Brain-Tumor-Detection-using-MLflow-DVC.mlflow/api/2.0/mlflow-artifacts/artifacts/c15e2345ab264f748eed5d704551368d/80268a5172a74061930562d5a30776e9/artifacts/model/data/model/variables/variables.data-00000-of-00001]


Successfully registered model 'Resnet50'.
2023/10/03 14:40:36 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: Resnet50, version 1
Created version '1' of model 'Resnet50'.
